# `q_net`

`-` 데이터를 모아보자.

In [193]:
current_states = collections.deque(maxlen=50) 
actions = collections.deque(maxlen=50) 
next_states = collections.deque(maxlen=50) 
rewards = collections.deque(maxlen=50) 
terminations = collections.deque(maxlen=50) 

current_state, _ = env.reset()
for t in range(500): 
    action = env.action_space.sample()
    next_state, reward, terminated, _, _ = env.step(action)

    current_states.append(current_state) 
    actions.append(action)
    next_states.append(next_state)
    rewards.append(reward)
    terminations.append(terminated) 
    
    current_state = next_state 
    if terminated: break 

`-` 이전코드에서 아래에 대응하는 부분을 잘 구현하면 된다.

``` python
## 1. q[x,y,a] 
agent.q = np.zeros([4,4,4])  

## 2. q_estimate  
x,y = agent.current_state
xx,yy = agent.next_state
a = agent.action 
q_estimated = agent.q[x,y,a] 

## 3. q_realistic = reward + 0.99 * q_future
if agent.terminated:
    q_realistic = agent.reward
else:
    q_future = q[xx,yy,:].max()
    q_realistic = agent.reward + 0.99 * q_future

## 4. q_estimate 와 q_realistic 를 비슷하게 만들어주는 역할을 하는 코드 
diff = q_realistic - q_estimated 
agent.q[x,y,a] = q_estimated + 0.05 * diff
```

`1`. q_net를 설정

In [201]:
q_net = torch.nn.Sequential(
    torch.nn.Linear(8,128), # 8개의 상태공간 
    torch.nn.ReLU(),
    torch.nn.Linear(128,64), 
    torch.nn.ReLU(),
    torch.nn.Linear(64,32), 
    torch.nn.ReLU(),
    torch.nn.Linear(32,4) # 4개의 action값들 
)

In [202]:
torch.tensor(current_state)

In [192]:
q_net(torch.tensor(current_state))

`1`. q_net를 설정 (배치버전)

In [199]:
torch.tensor(current_states)

/tmp/ipykernel_141614/982066375.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/torch/csrc/utils/tensor_new.cpp:245.)
  torch.tensor(current_states)

In [200]:
torch.tensor(np.array(current_states)).shap